In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import time
#install pytrends
#!pip install pytrends
from pytrends.request import TrendReq

In [153]:
# scale data based on which year and country we're interested in
# Idea: Correct the data by scaling it here and write a new csv-file with this new data, before using it further.

def get_historical_dataframe(keyword, timeframe, geo=''):
    kw_list = [keyword]
    pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1, requests_args={'verify':False})
    pytrend.build_payload(kw_list, timeframe=timeframe, geo=geo)
    
    # next we make the dataframe for this word over the period
    BP = pytrend.interest_over_time()
    if BP.empty:
        BP = pd.DataFrame([1,1],columns=[keyword])

    return BP

def scale_data(df,years,keyword,geo=''):
    '''
    df: dataframe of one country
    years: years we want to stitch together (can get this from df)
    geo: code name of country
    column: which column we want to scale. 2 for first keyword
    returns a dataframe where the specified column is scaled appropriately
    '''
    
    # get years from df
    #    
    df_nor.reset_index(inplace=True,drop=True)
    
    # get dates where we patch
    patch_dates_from = []
    patch_dates_to = []
    for year in years:
        patch_dates_from.append(f"{year}-06-30")
        patch_dates_to.append(f"{year}-07-01")
        patch_dates_from.append(f"{year}-12-31")
        patch_dates_to.append(f"{year+1}-01-01")
    patch_dates_from.pop()
    patch_dates_to.pop()
    
    for i in range(len(patch_dates_from)):
        # prepare data
        
        i_date_from = df_nor.index[df_nor["date"]==patch_dates_from[i]].tolist()[0]
        i_date_to = df_nor.index[df_nor["date"]==patch_dates_to[i]].tolist()[0]
        date_from = df_nor["date"][i_date_from]
        date_to = df_nor["date"][i_date_to]
        timeframe = date_from + " " + date_to
        #print(timeframe)
        P = [df_nor[keyword][i_date_from],df_nor[keyword][i_date_to]]
        #print(P)
        
        BP = get_historical_dataframe(keyword,timeframe,geo)
        print(BP)
        BP_vals = [BP[keyword][0],BP[keyword][1]]
        
        # actually do the scaling
        s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
        #print(df[keyword][0:])
        df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
        
    df[keyword] = 100*df[keyword]/df[keyword].max()
    
    return df
        

'''
filename = "bigboy.csv"
df = pd.read_csv(filename)


YEARS = [2010]#, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_nor = df.loc[df['country'] == "Norway"]
new_df = scale_data(df_nor,YEARS,"Facebook")
print(len(df.loc[df['country'] == "Norway"]))
df.loc[df['country'] == "Norway"] = new_df
print(new_df)
print(len(df.loc[df['country'] == "Norway"]))
# df.loc[df['country'] == "Norway"] = new_df => overwrites only for Norway. 
# Can loop over each country and search term to make new csv file
'''

'\nfilename = "bigboy.csv"\ndf = pd.read_csv(filename)\n\n\nYEARS = [2010]#, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]\ndf_nor = df.loc[df[\'country\'] == "Norway"]\nnew_df = scale_data(df_nor,YEARS,"Facebook")\nprint(len(df.loc[df[\'country\'] == "Norway"]))\ndf.loc[df[\'country\'] == "Norway"] = new_df\nprint(new_df)\nprint(len(df.loc[df[\'country\'] == "Norway"]))\n# df.loc[df[\'country\'] == "Norway"] = new_df => overwrites only for Norway. \n# Can loop over each country and search term to make new csv file\n'

In [151]:
def _load_unavailable_countries(filename: str):
    unavailable_countries = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            unavailable_countries.append(line)
    return unavailable_countries

def load_countries(filename: str, ignore: str):
    unavailable_countries = _load_unavailable_countries(ignore)
    valid_countries = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            line_list = line.split(",")
            code = line_list[0]
            country = line_list[1]
            if country not in unavailable_countries:
                valid_countries.append((code, country))
    return valid_countries

In [148]:
filename = "bigboy.csv"
df = pd.read_csv(filename)
df.reset_index(drop=True)
to_filename = "scaled_data.csv"

YEARS = [2010]
search_term = "Facebook"
country = "Norway"

country_df = df.loc[df["country"] == country]

scale_data(country_df,YEARS,search_term,"NO")

# before inserting the changed values into the original dataframe, make sure to match indexes
country_df.index = df.loc[df['country'] == country].index
# insert into dataframe
df.loc[df["country"] == country] = country_df

df.to_csv(to_filename)

C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\conne

            Facebook  isPartial
date                           
2010-06-30       100      False
2010-07-01        99      False
            Facebook  isPartial
date                           
2010-06-30       100      False
2010-07-01        99      False


<ipython-input-146-c3a943023eb7>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
C:\Users\usbb\anaconda3\lib\site-packages\pandas\core\series.py:998: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-148-358dce380bd1>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scale_data(country_df,YEARS,search_term,"NO")
<ipython-input-146-c3a943023e

'Ebola'

In [154]:
filename = "bigboy.csv"
to_filename = "scaled_data.csv"

valid_countries = load_countries("docs/countries.txt","docs/ignore.txt")
countries = df["country"].unique()
search_terms = df.columns[2:]

YEARS = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df = pd.read_csv(filename)
i = 0
for search_term in search_terms:
    for country in countries:
        geo = ""
        for geocode,check_country in valid_countries:
            if country == check_country:
                geo = geocode
                print(country,geo)
        # patch data using search_term,country,geocode
        '''
        How: 
        make new df containing only the data for country
        scale this data
        '''
        print(i)
        country_df = df.loc[df["country"] == country]

        scale_data(country_df,YEARS,search_term,geo)

        # before inserting the changed values into the original dataframe, make sure to match indexes
        country_df.index = df.loc[df['country'] == country].index
        # insert into dataframe
        df.loc[df["country"] == country] = country_df
    pf.to_csv(f"scale/try_{i}_{to_filename}")
    i += 1



'''
Rate Limit is not publicly known, let me know if you have a consistent estimate
* One user reports that 1,400 sequential requests of a 4 hours timeframe got them to the limit. (Replicated on 2 networks)
* It has been tested, and 60 seconds of sleep between requests (successful or not) appears to be the correct amount once you reach the limit.
time.sleep(60)
'''

Ecuador EC
0


C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\usbb\anaconda3\lib\site-packages\urllib3\conne

   Ebola
0      1
1      1


<ipython-input-153-499dd9447dc7>:57: RuntimeWarning: invalid value encountered in double_scalars
  s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
<ipython-input-153-499dd9447dc7>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
C:\Users\usbb\anaconda3\lib\site-packages\pandas\core\series.py:998: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-154-b99c0e6f88ed>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

   Ebola
0      1
1      1


...

            Ebola  isPartial
date                        
2017-12-31      0      False
2018-01-01    100      False


<ipython-input-153-499dd9447dc7>:57: RuntimeWarning: divide by zero encountered in long_scalars
  s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
<ipython-input-153-499dd9447dc7>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
C:\Users\usbb\anaconda3\lib\site-packages\pandas\core\series.py:998: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-154-b99c0e6f88ed>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

            Ebola  isPartial
date                        
2018-06-30      0      False
2018-07-01    100      False


<ipython-input-153-499dd9447dc7>:57: RuntimeWarning: divide by zero encountered in long_scalars
  s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
<ipython-input-153-499dd9447dc7>:57: RuntimeWarning: invalid value encountered in double_scalars
  s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
<ipython-input-153-499dd9447dc7>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
C:\Users\usbb\anaconda3\lib\site-packages\pandas\core\series.py:998: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_with(key, value)
<ipython-input-154-b99c0e6f88ed>:27: SettingWithCopyWarning: 
A value is try

RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22Ebola%22%2C+%22time%22%3A+%222018-12-31+2019-01-01%22%2C+%22geo%22%3A+%22ZA%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))

In [139]:
df

,date,country,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
0,2010-01-01,Ecuador,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.0,75.0,0.0,0.0,0.0
1,2010-01-01,Estonia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,52.0,94.0,0.0,16.0,0.0
2,2010-01-01,Colombia,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,77.0,46.0,0.0,0.0,0.0
3,2010-01-01,Venezuela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.0,24.0,0.0,2.0,0.0
4,2010-01-01,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,67.0,41.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278279,2019-12-31,Peru,7.0,0.0,7.0,0.0,14.0,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,85.0,52.0,12.0,3.0,0.0
278280,2019-12-31,New Zealand,0.0,0.0,0.0,2.0,0.0,7.0,0.0,3.0,...,5.0,3.0,5.0,0.0,0.0,89.0,60.0,26.0,7.0,4.0
278281,2019-12-31,Netherlands,0.0,0.0,1.0,0.0,2.0,24.0,0.0,8.0,...,15.0,10.0,10.0,0.0,5.0,89.0,79.0,34.0,15.0,3.0
278282,2019-12-31,Colombia,2.0,0.0,2.0,0.0,2.0,5.0,0.0,5.0,...,74.0,0.0,25.0,0.0,49.0,83.0,72.0,22.0,7.0,1.0


In [3]:
def get_corr_matrix(filename,category,glob=False):
    '''
    filename: Trends/filename (remember .csv). ex: "db_0.csv"
    category: ex "Facebook", "YouTube"
    returns list of all countries involved (all names of columns/rows) and correlation matrix 
    '''
    df = pd.read_csv(f"{filename}")
    df["date"]= pd.to_datetime(df["date"])
    df.dropna(inplace=True)
    
    # Keep country and Facebook
    df = df[["country", category]]
    
    # Get a list of countries
    countries = df["country"].unique()
    
    # Create a new DataFrame 
    new_df = pd.DataFrame()
    for country in countries:
        tmp = df[df["country"] == country]#
        tmp = tmp.drop(columns=["country"])
        tmp = tmp.rename(columns={category: country})
        new_df = pd.concat([new_df.reset_index(drop=True), tmp.reset_index(drop=True)], axis=1)
    
    if glob:
        # Remove Global, and insert it at the front
        new_df["Global"] = new_df.sum(axis=1) / new_df.shape[1]
        GLOB = new_df.pop('Global')
        new_df.insert(0, "Global", GLOB)
    
    new_df = new_df.sort_index(axis=1)
        
    CORRELATION_MATRIX = new_df.corr()
    
    return CORRELATION_MATRIX.columns, CORRELATION_MATRIX
    
cnt,mtr = get_corr_matrix("bigboy.csv","Facebook")

In [5]:
# get categories for each file
def get_categories(filename):
    '''
    filename: filename (.csv)
    returns categories as a 2D-list where each
    inner list refers to a filename
    '''    
    df = pd.read_csv(f"{filename}")
    df["date"]= pd.to_datetime(df["date"])
    df.dropna(inplace=True)
    categories = df.columns[2:]
    return categories

'Ebola'

In [6]:
# get corr matrix
# prepare to get corr matrices
filename = "bigboy.csv"
categories = get_categories(filename)
#print(filename,categories)

# get it for each category/term
corr_matrices = []
for cat in categories:
    countries, corr_matrix = get_corr_matrix(filename,cat)
    corr_matrices.append([cat,countries,corr_matrix])
corr_matrices

[['Ebola',
  Index(['Afghanistan', 'Albania', 'Argentina', 'Australia', 'Austria',
         'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia & Herzegovina',
         'Brazil', 'Cameroon', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia',
         'Cuba', 'Czechia', 'Denmark', 'Ecuador', 'Egypt', 'Estonia', 'Finland',
         'France', 'Georgia', 'Germany', 'Greece', 'Hong Kong', 'Hungary',
         'Iceland', 'India', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
         'Japan', 'Kenya', 'Latvia', 'Lebanon', 'Libya', 'Lithuania',
         'Luxembourg', 'Mexico', 'Mongolia', 'Morocco', 'Netherlands',
         'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Paraguay', 'Peru',
         'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Saudi Arabia',
         'Serbia', 'Singapore', 'Slovakia', 'Slovenia', 'South Africa',
         'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia',
         'Turkey', 'Ukraine', 'United Kingdom', 'United States', 'Uruguay',


In [7]:
avg_corr_matrix = corr_matrices[0][2].copy()

for col in avg_corr_matrix.columns:
    avg_corr_matrix[col].values[:] = 0
print(avg_corr_matrix)
    
for i in range(50):
    i_matrix = corr_matrices[i][2].copy().fillna(1)
    avg_corr_matrix += i_matrix
print(avg_corr_matrix)

avg_corr_matrix /= 50
print(avg_corr_matrix)

                Afghanistan  Albania  Argentina  Australia  Austria  \
Afghanistan             0.0      0.0        0.0        0.0      0.0   
Albania                 0.0      0.0        0.0        0.0      0.0   
Argentina               0.0      0.0        0.0        0.0      0.0   
Australia               0.0      0.0        0.0        0.0      0.0   
Austria                 0.0      0.0        0.0        0.0      0.0   
...                     ...      ...        ...        ...      ...   
Ukraine                 0.0      0.0        0.0        0.0      0.0   
United Kingdom          0.0      0.0        0.0        0.0      0.0   
United States           0.0      0.0        0.0        0.0      0.0   
Uruguay                 0.0      0.0        0.0        0.0      0.0   
Venezuela               0.0      0.0        0.0        0.0      0.0   

                Bangladesh  Belarus  Belgium  Bolivia  Bosnia & Herzegovina  \
Afghanistan            0.0      0.0      0.0      0.0               

In [11]:
def convert_matrix(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    converts matrix to a matrix where all values > treshold gets replaced with 1 and the rest 0
    '''
    matrix = corr_matrix.copy()
    matrix[matrix>=treshold] = 1
    matrix[matrix<treshold] = 0
    
    return matrix

def make_groups_from_matrix(corr_matrix, treshold, used_countries):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    used_countries: countries you don't want to be grouped up. Useful if we already have a group
    returns list of lists, where each list is a group
    '''
    matrix = convert_matrix(corr_matrix,treshold)
    #print(corr_matrix,treshold,used_countries)
    groups = []
    countries = matrix.columns
    for c in countries:
        if c not in used_countries:
            # if country not visited before, make a new group and make sure all correlated countries are added
            group = [c]
            used_countries.append(c)             
            
            for ct in group:
                # make list of countries that ct correlates with
                # append all countries that has not been visited before to group and used_countries
                ct_corr = matrix[ct][matrix[ct]==1].index.values
                for ctr in ct_corr:
                    if ctr not in used_countries:
                        used_countries.append(ctr)
                        group.append(ctr)
            
            
            groups.append(group)
    return groups

In [12]:
# Method 1: Is for sure making larger groups. Might look nice, but doesn't necessarily say that much
# Idea: Make groups with high treshold, then of the remaining countries, use lower treshold, until reach 0 as treshold

def groupifier(corr_matrix,start_treshold=0.9,skip_treshold=0.1):
    '''
    returns groups in a list, and groups on the form: [[treshold,[groups]]xG] (G groups)
    '''
    groups = []
    groups_treshold = []
    used_countries = []
    steps = int(start_treshold/skip_treshold+1)
    for treshold in np.linspace(start_treshold,0,steps):
        gr = make_groups_from_matrix(corr_matrix, treshold, used_countries[:])
        # copy group
        gr_copy = gr[:]
        for g in gr_copy:
            #print(g)
            if len(g)==1:
                gr.remove(g)
            else:
                used_countries+=g
                groups.append(g)
        if len(gr):
            groups_treshold.append([round(treshold,1),gr])

    return groups,groups_treshold
        
comat = corr_matrices[0][2]
#print(comat)
groups, groups_treshold = groupifier(comat,0.9,0.1)
print(groups)

[['India', 'South Africa'], ['Austria', 'Germany', 'Switzerland', 'Netherlands'], ['Canada', 'United Kingdom', 'United States', 'Ireland'], ['Australia', 'New Zealand'], ['Belgium', 'France', 'Italy', 'Kenya', 'Nigeria', 'Norway', 'Pakistan', 'Venezuela', 'Mexico', 'Spain', 'Denmark', 'Egypt', 'Lebanon', 'Singapore', 'Bolivia', 'Brazil', 'Peru', 'Finland', 'Sweden', 'Colombia', 'Ecuador'], ['Cameroon', 'Hong Kong', 'Japan', 'Portugal', 'Saudi Arabia', 'Morocco', 'Iran'], ['Argentina', 'Chile', 'Greece', 'Paraguay', 'Turkey', 'Uruguay', 'Israel', 'Tunisia', 'Bangladesh', 'Bosnia & Herzegovina', 'Croatia', 'Estonia', 'Latvia', 'Lithuania', 'Luxembourg', 'Qatar', 'Russia', 'Serbia', 'South Korea', 'Cuba', 'Ukraine', 'Iceland', 'Iraq', 'Slovenia'], ['Afghanistan', 'Libya'], ['Albania', 'Georgia', 'Mongolia'], ['China', 'Poland', 'Slovakia', 'Czechia', 'Hungary', 'Romania', 'Thailand']]


In [16]:
# Method 2: Doesn't necessarily give big groups, but makes sure groups are well defined
# Meaning: All countries in each group correlates at least treshold to all other countries in that group

def groupifier2(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix
    treshold: number between 0 and 1
    returns groups where all countries in each group correlate
        at least treshold with each other country in the group
    '''
    # make a list containing a list with each country
    groups = [corr_matrix.columns.values.tolist()]
    # idea: split group into sub-groups until each sub-group fullfills criteria
    #print(len(groups[0]))
    while True:
        new_groups = []
        for group in groups:
            # find countries that correlate least
            country1 = ""
            country2 = ""
            least_treshold = 1
            for c1 in group:
                for c2 in group:
                    if corr_matrix[c1][c2] < least_treshold:
                        country1 = c1
                        country2 = c2
                        least_treshold = corr_matrix[c1][c2]
            if least_treshold < treshold:
                break_loop = False
                g1 = [country1]
                g2 = [country2]
                for country in group:
                    if country != country1 and country != country2:
                        if corr_matrix[country1][country] > corr_matrix[country2][country]:
                            g1.append(country)
                        else:
                            g2.append(country)
                new_groups.append(g1)
                new_groups.append(g2)
            else:
                new_groups.append(group)
                       
        # if no changes are done, break loop. if not, 
        if new_groups==groups:
            break
        else:
            groups = new_groups
    
    return groups


comat = corr_matrices[46][2]
groups_treshold = groupifier2(comat,0.7)
print(groups_treshold)
print(len(groups_treshold))

[['Netherlands', 'Portugal'], ['Poland', 'Brazil', 'Romania'], ['Paraguay', 'Hungary'], ['Bolivia', 'Mexico', 'Peru'], ['Lithuania'], ['Spain'], ['Estonia'], ['Argentina'], ['Ecuador'], ['Venezuela'], ['Qatar'], ['Belgium', 'Chile', 'Colombia', 'Germany', 'Italy'], ['China'], ['Iraq'], ['Greece', 'Turkey', 'Bangladesh', 'Cameroon', 'Cuba', 'Morocco', 'Nigeria'], ['Pakistan'], ['Kenya'], ['Mongolia'], ['India', 'Thailand', 'Egypt', 'Saudi Arabia'], ['Iran'], ['Libya'], ['Afghanistan'], ['Georgia'], ['Albania', 'Israel', 'Bosnia & Herzegovina', 'Lebanon', 'Norway', 'Tunisia'], ['South Africa', 'Croatia', 'United States'], ['New Zealand', 'Hong Kong', 'Australia', 'Singapore', 'South Korea', 'United Kingdom', 'Uruguay'], ['Iceland', 'Czechia', 'Denmark', 'Sweden'], ['Belarus'], ['Japan'], ['Russia'], ['Latvia'], ['Ukraine'], ['Slovenia', 'Slovakia'], ['Serbia'], ['Ireland', 'Austria', 'Canada', 'Finland', 'France', 'Luxembourg', 'Switzerland']]
35


In [23]:
gr1 = make_groups_from_matrix(avg_corr_matrix, 0.5, [])

gr2, gr2_tresh = groupifier(avg_corr_matrix)

gr3 = groupifier2(avg_corr_matrix,0.5)

print(len(gr1),gr1,"\n\n\n",len(gr2),gr2_tresh,"\n\n\n",len(gr3),gr3)

75 [['Afghanistan'], ['Albania'], ['Argentina'], ['Australia'], ['Austria'], ['Bangladesh'], ['Belarus'], ['Belgium'], ['Bolivia'], ['Bosnia & Herzegovina'], ['Brazil'], ['Cameroon'], ['Canada', 'United Kingdom', 'United States'], ['Chile'], ['China'], ['Colombia'], ['Croatia'], ['Cuba'], ['Czechia'], ['Denmark'], ['Ecuador'], ['Egypt'], ['Estonia'], ['Finland'], ['France'], ['Georgia'], ['Germany'], ['Greece'], ['Hong Kong'], ['Hungary'], ['Iceland'], ['India'], ['Iran'], ['Iraq'], ['Ireland'], ['Israel'], ['Italy'], ['Japan'], ['Kenya'], ['Latvia'], ['Lebanon'], ['Libya'], ['Lithuania'], ['Luxembourg'], ['Mexico'], ['Mongolia'], ['Morocco'], ['Netherlands'], ['New Zealand'], ['Nigeria'], ['Norway'], ['Pakistan'], ['Paraguay'], ['Peru'], ['Poland'], ['Portugal'], ['Qatar'], ['Romania'], ['Russia'], ['Saudi Arabia'], ['Serbia'], ['Singapore'], ['Slovakia'], ['Slovenia'], ['South Africa'], ['South Korea'], ['Spain'], ['Sweden'], ['Switzerland'], ['Thailand'], ['Tunisia'], ['Turkey'], ['

In [32]:
for i in range(len(gr2_tresh)):
    for j in range(len(gr2_tresh[i][1])):
        df = pd.DataFrame(gr2_tresh[i][1][j])
        df.columns = [gr2_tresh[i][0]]
        df.to_csv(f"groups/treshold_{gr2_tresh[i][0]}_group_{j+1}.csv",index=False)
